In [1]:
import boto3
import pandas as pd
import psycopg2
import json

In [2]:
import configparser
config = configparser.ConfigParser()
config.read_file(open(r"C:\Users\hasan\OneDrive\Desktop\cluster.config"))

In [3]:
KEY = config.get("AWS","KEY")
SECRET = config.get("AWS","SECRET")
DWH_CLUSTER_TYPE = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE = config.get("DWH" , "DWH_NODE_TYPE")


DWH_CLUSTER_IDENTIFIER = config.get("DWH", "DWH_CLUSTER_IDENTIFIER" )
DWH_DB = config.get("DWH", "DWH_DB" )
DWH_DB_USER = config.get("DWH" , "DWH_DB_USER")
DWH_DB_PASSWORD = config.get("DWH" , "DWH_DB_PASSWORD")

DWH_PORT = config.get("DWH" , "DWH_PORT")
DWH_IAM_ROLE_NAME = config.get("DWH" , "DWH_IAM_ROLE_NAME")




(DWH_DB_USER , DWH_DB_PASSWORD , DWH_DB)

('awsuser', 'Passw0rd123', 'myfirstdb')

In [4]:
pd.DataFrame({"Param":
              ["DWH_CLUSTER_TYPE","DWH_NUM_NODES" , "DWH_NODE_TYPE" , "DWH_CLUSTER_IDENTIFIER" , "DWH_DB" ,"DWH_DB_USER" , "DWH_DB_PASSWORD" , "DWH_PORT" , "DWH_IAM_ROLE_NAME"],
              "Value":
              [DWH_CLUSTER_TYPE ,DWH_NUM_NODES , DWH_NODE_TYPE , DWH_CLUSTER_IDENTIFIER , DWH_DB , DWH_DB_USER ,  DWH_DB_PASSWORD , DWH_PORT ,DWH_IAM_ROLE_NAME ]
    
})

,Param,Value
0,DWH_CLUSTER_TYPE,single-node
1,DWH_NUM_NODES,1
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,my-first-redshift
4,DWH_DB,myfirstdb
5,DWH_DB_USER,awsuser
6,DWH_DB_PASSWORD,Passw0rd123
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,redshift-s3-access


In [5]:
ec2 = boto3.resource('ec2',
                    region_name="us-east-1",
                    aws_access_key_id = KEY,
                     aws_secret_access_key = SECRET   
                    )

In [6]:
s3 = boto3.resource('s3',
                    region_name="us-east-1",
                    aws_access_key_id = KEY,
                     aws_secret_access_key = SECRET   
                    )
iam = boto3.client('iam',
                    region_name="us-east-1",
                    aws_access_key_id = KEY,
                     aws_secret_access_key = SECRET   
                    )
redshift = boto3.client('redshift',
                    region_name="us-east-1",
                    aws_access_key_id = KEY,
                     aws_secret_access_key = SECRET   
                    )

In [7]:
bucket = s3.Bucket("hasan8198")
log_data_files = [filename.key for filename in bucket.objects.filter(Prefix='')]
log_data_files

['allevents_pipe.txt',
 'allusers_pipe.txt',
 'category_pipe.txt',
 'date2008_pipe.txt',
 'listings_pipe.txt',
 'sales_tab.txt',
 'venue_pipe.txt']

In [11]:
#roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']
roleArn ='arn:aws:iam::816801429483:role/redshift-s3-access'

In [12]:
try:
    response = redshift.create_cluster(
    ClusterType = DWH_CLUSTER_TYPE,
    NodeType = DWH_NODE_TYPE,
    
    DBName = DWH_DB,
    ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
    MasterUsername = DWH_DB_USER,
    MasterUserPassword = DWH_DB_PASSWORD,
   
    #Roles (For s3 access)
    IamRoles=[roleArn]
        
    )
except Exception as e:
    print(e)

An error occurred (AccessDenied) when calling the CreateCluster operation: User: arn:aws:iam::816801429483:user/xetra_user is not authorized to perform: iam:PassRole on resource: arn:aws:iam::816801429483:role/redshift-s3-access because no identity-based policy allows the iam:PassRole action


In [15]:
redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Ckusters'][0]

ClusterNotFoundFault: An error occurred (ClusterNotFound) when calling the DescribeClusters operation: Cluster my-first-redshift not found.

In [17]:
def prettyRedShiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier","NodeType","ClusterStatus","MasterUsername","DBName","Endpoint","VpcId"]
    x = [(k,v) for k,v in props.items() if k in keysToShow]
    return db.DataFrame(data=x, columns=["key" ,"Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Ckusters'][0]
prettyRedShiftProps(myClusterProps)

ClusterNotFoundFault: An error occurred (ClusterNotFound) when calling the DescribeClusters operation: Cluster my-first-redshift not found.

In [19]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
DB_NAME = myClusterProps['DBName']
DB_USER = myClusterProps['MasterUsername']

NameError: name 'myClusterProps' is not defined

In [20]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
        GroupName = defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort = int(DWH_PORT),
        ToPort = int(DWH_PORT)
    )
except Exception as e:
    print(e)

name 'myClusterProps' is not defined


In [21]:
try:
    conn = psycopg2.connect(host=DWH_ENDPOINT , dbname = DB_NAME, user = DB_USER , password = "Passw0rd123" , port = 5439)
except psycopg2.Error as e:
    print("Error: could not make conenction ot the postgre database")
    print(e)

conn.set_session(autocommit=True)

NameError: name 'DWH_ENDPOINT' is not defined

In [22]:
try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error: could not create cursor")
    print(e)


NameError: name 'conn' is not defined

In [23]:
try:
    cur.execute("""create table users(
    userid integer not null distkey sortkey,
    username char(8),
    firstname varchar(30),
    lastname varchar(30),
    city varchar(30),
    state varchar(2),
    email varchar(100),
    phone char(14),
    likesports boolean,
    liketheatre boolean,
    likeconcerts boolean,
    likejazz boolean,
    likerock boolean,
    likeopera boolean,
    likerock boolean,
    likevegas boolean,
    likebroadway boolean,
    likemusicals boolean
    );  
    """)
except psycopg2.Error as e:
    print("Error: could not create table")
    print(e)


NameError: name 'cur' is not defined

In [24]:
try:
    cur.execute("""create table venue(
       venueid smallint not null distkey sortkey,
       venuename varchar(100),
       venuecity varchar(30),
       venuestate char(2),
       venueseats integer
    );  
    """)
except psycopg2.Error as e:
    print("Error: could not create table")
    print(e)


NameError: name 'cur' is not defined

In [25]:
try:
    cur.execute("""create table category(
       catid smallint not null distkey sortkey,
       catgroup varchar(10),
       catname varchar(10),
       catdesc varchar(50)
    ); 
    
    create table date(
       dateid smallint not null distkey sortkey,
       caldate data not null,
       day character(3) not null,
       week smallint not null,
       month character(5) not null,
       qtr character(5) not null,
       year smallint not null,
       holiday boolean default('N'));
    
    
    create table event(
       eventid integer not null distkey ,
       venueid smallint not null,
       catid smallint not null,
       dateid smallint not null sortkey,
       eventname varchar(200),
       starttime timestamp);
       
       
      
    create table listing(
       listid integer not null distkey ,
       sellerid integer not null,
       eventid integer not null,
       dateid smallint not null sortkey,
       numtickets smallint not null,
       priceperticket decimal(8,2),
       totalprice decimal(8,2),
       listtime timestamp);
    
    
    """)
except psycopg2.Error as e:
    print("Error: could not create table")
    print(e)


NameError: name 'cur' is not defined

In [26]:
try:
    cur.execute("""
        copy users from 's3://hasan8198/allusers_pipe.txt'
        credentials 'aws_iam_role=arn:aws:iam::816801429483:role/redshift-s3-access'
        dalimiter '|'
        region 'us-east-1'
    
        copy venue from 's3://hasan8198/allusers_pipe.txt'
        credentials 'aws_iam_role=arn:aws:iam::816801429483:role/redshift-s3-access'
        dalimiter '|'
        region 'us-east-1'
    
        copy category from 's3://hasan8198/allusers_pipe.txt'
        credentials 'aws_iam_role=arn:aws:iam::816801429483:role/redshift-s3-access'
        dalimiter '|'
        region 'us-east-1'
        
        copy date from 's3://hasan8198/allusers_pipe.txt'
        credentials 'aws_iam_role=arn:aws:iam::816801429483:role/redshift-s3-access'
        dalimiter '|'
        region 'us-east-1'
        
        copy event from 's3://hasan8198/allusers_pipe.txt'
        credentials 'aws_iam_role=arn:aws:iam::816801429483:role/redshift-s3-access'
        dalimiter '|'
        region 'us-east-1'
        
        copy listing from 's3://hasan8198/allusers_pipe.txt'
        credentials 'aws_iam_role=arn:aws:iam::816801429483:role/redshift-s3-access'
        dalimiter '|'
        region 'us-east-1'
    """)
    
except psycopg2.Error as e:
    print("Error: could not create table")
    print(e)


NameError: name 'cur' is not defined

In [27]:
try:
    cur.execute("""
        select * from users;
    """)
    
except psycopg2.Error as e:
    print("Error: could not create table")
    print(e)



NameError: name 'cur' is not defined

In [28]:
row = cur.fetchone()
print(row)


NameError: name 'cur' is not defined

In [29]:
try:
    conn.close()
    
except psycopg2.Error as e:
    print("Error: could not create table")
    print(e)


NameError: name 'conn' is not defined

In [ ]:
redshift.delete_cluster(ClusterIdentifier = DWH_CLUSTER_IDENTIFIER , SkipFinalClusterSnapshot = True)